# Data Validation

In the previous notebook, two pins were saved:

- City of Chicago - Business License Data (RAW): `chicago-business-license-data`
- ity of Chicago - Food Inspection Data (RAW): `chicago-food-inspection-data`

## Setup

In [1]:
import pandas as pd
import pandera as pa
import pins
# from ydata_profiling import ProfileReport

In [2]:
pd.options.display.max_columns = 999

In [3]:
# Set up the board
board = pins.board_connect()
user_name = "sam.edwardes"

## Data set (1): Business License Data

<https://data.cityofchicago.org/Community-Economic-Development/Business-Licenses/r5kz-chrr>

In [4]:
pin_name = f"{user_name}/chicago-business-license-data-raw"
business_license_raw = board.pin_read(pin_name)
business_license_raw

,id,license_id,account_number,site_number,legal_name,doing_business_as_name,address,city,state,zip_code,ward,precinct,ward_precinct,police_district,license_code,license_description,business_activity_id,business_activity,license_number,application_type,application_created_date,application_requirements_complete,payment_date,conditional_approval,license_start_date,expiration_date,license_approved_for_issuance,date_issued,license_status,license_status_change_date,ssa,latitude,longitude,location
0,1000000-20020221,1000000,200001,1,MARK BOSTON,COLORS IN MOTION,6421 N DAMEN AVE,CHICAGO,IL,60645,50.0,28.0,50-28,24.0,1011,Home Repair,NaN,NaN,1000000,ISSUE,2000-06-19T00:00:00.000,2002-02-15T00:00:00.000,2002-02-15T00:00:00.000,N,2002-02-21T00:00:00.000,2002-11-15T00:00:00.000,2002-02-21T00:00:00.000,2002-02-22T00:00:00.000,AAI,NaN,NaN,41.998514,-87.680011,"\n, \n(41.99851437112669, -87.68001090539342)"
1,1000049-20010816,1162772,200068,1,ANTONIA CASTREJON,ILLUSIONS HAIR DESIGN,3800 W DIVERSEY AVE,CHICAGO,IL,60647,31.0,999.0,31-999,25.0,1010,Limited Business License,NaN,NaN,1000049,RENEW,NaN,2001-06-25T00:00:00.000,2001-08-20T00:00:00.000,N,2001-08-16T00:00:00.000,2002-08-15T00:00:00.000,2001-08-20T00:00:00.000,2002-04-30T00:00:00.000,AAI,NaN,NaN,41.931960,-87.722150,"\n, \n(41.931960332638006, -87.72215036594574)"
2,1000049-20020516,1233615,10141,2,"PEPE""S RETAIL MEATS, INC.",PEREZ MEXICAN FOOD,853-855 W RANDOLPH ST 1ST,CHICAGO,IL,60607,27.0,1.0,27-1,12.0,1006,Retail Food Establishment,775.0,Retail Sales of Perishable Foods,1000049,RENEW,NaN,2002-03-27T00:00:00.000,2002-04-17T00:00:00.000,N,2002-05-16T00:00:00.000,2003-05-15T00:00:00.000,2002-04-17T00:00:00.000,2002-04-18T00:00:00.000,AAI,NaN,NaN,41.884261,-87.649534,"\n, \n(41.88426142200001, -87.6495341312589)"
3,1000049-20020816,1265665,200068,1,ANTONIA CASTREJON,ILLUSIONS HAIR DESIGN,3800 W DIVERSEY AVE,CHICAGO,IL,60647,31.0,999.0,31-999,25.0,1010,Limited Business License,NaN,NaN,1000049,RENEW,NaN,2002-06-28T00:00:00.000,2002-08-13T00:00:00.000,N,2002-08-16T00:00:00.000,2003-08-15T00:00:00.000,2002-08-13T00:00:00.000,2002-08-14T00:00:00.000,AAI,NaN,NaN,41.931960,-87.722150,"\n, \n(41.931960332638006, -87.72215036594574)"
4,1000049-20030516,1342680,10141,2,"PEPE""S RETAIL MEATS, INC.",PEREZ MEXICAN FOOD,853-855 W RANDOLPH ST 1ST,CHICAGO,IL,60607,27.0,1.0,27-1,12.0,1006,Retail Food Establishment,775.0,Retail Sales of Perishable Foods,1000049,RENEW,NaN,2003-03-25T00:00:00.000,2003-04-17T00:00:00.000,N,2003-05-16T00:00:00.000,2004-05-15T00:00:00.000,2003-04-17T00:00:00.000,2003-04-18T00:00:00.000,AAI,NaN,NaN,41.884261,-87.649534,"\n, \n(41.88426142200001, -87.6495341312589)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1000966-20020216,1215943,200860,1,CHAUNDEL D SAMMARCO,CHAUNDEL D.SAMMARCO,3705 WONDERLAKE RD,WONDER LAKE,IL,60007,NaN,NaN,NaN,NaN,1604,"Peddler, non-food",NaN,NaN,1000966,RENEW,NaN,2001-12-20T00:00:00.000,2002-02-15T00:00:00.000,N,2002-02-16T00:00:00.000,2003-02-15T00:00:00.000,NaN,2003-02-20T00:00:00.000,AAI,NaN,NaN,NaN,NaN,NaN
996,1000966-20030216,1334986,200860,1,CHAUNDEL D SAMMARCO,CHAUNDEL D.SAMMARCO,3705 WONDERLAKE RD,WONDER LAKE,IL,60007,NaN,NaN,NaN,NaN,1604,"Peddler, non-food",NaN,NaN,1000966,RENEW,NaN,2003-02-20T00:00:00.000,2003-02-20T00:00:00.000,N,2003-02-16T00:00:00.000,2004-02-15T00:00:00.000,NaN,2003-12-22T00:00:00.000,AAI,NaN,NaN,NaN,NaN,NaN
997,1000966-20040216,1477145,200860,1,CHAUNDEL D SAMMARCO,CHAUNDEL D.SAMMARCO,3705 WONDERLAKE RD,WONDER LAKE,IL,60007,NaN,NaN,NaN,NaN,1604,"Peddler, non-food",NaN,NaN,1000966,RENEW,NaN,2004-02-17T00:00:00.000,2004-02-17T00:00:00.000,N,2004-02-16T00:00:00.000,2005-02-15T00:00:00.000,NaN,2004-02-17T00:00:00.000,AAI,NaN,NaN,NaN,NaN,NaN
998,1000966-20050216,1562532,200860,1,CHAUNDEL D SAMMARCO,CHAUNDEL D.SAMMARCO,3705 WONDERLAKE RD,WONDER LAKE,IL,60007,NaN,NaN,NaN,NaN,1604,"Peddler, non-food",NaN,NaN,1000966,RENEW,NaN,2004-1

**Tip:** Use multiple cursors in VS Code to easily edit many lines at the same time (<https://code.visualstudio.com/docs/getstarted/tips-and-tricks#_column-box-selection>).

In [12]:
business_license_schema = pa.DataFrameSchema({
    "id": pa.Column(str, coerce=True),
    "license_id": pa.Column(str, coerce=True, unique=True),
    "account_number": pa.Column(str, coerce=True),
    "site_number": pa.Column(str, coerce=True),
    "legal_name": pa.Column(str, coerce=True),
    "doing_business_as_name": pa.Column(str, coerce=True),
    "address": pa.Column(str, coerce=True),
    "city": pa.Column(str, coerce=True),
    "state": pa.Column(str, coerce=True, checks=[
        pa.Check.eq("IL")
    ]),
    "zip_code": pa.Column(str, coerce=True, checks=[
        pa.Check(lambda x: x.str.match(r'^\d{5}$').all())
    ]),
    "ward": pa.Column(str, coerce=True, nullable=True),
    "precinct": pa.Column(str, coerce=True),
    "ward_precinct": pa.Column(str, coerce=True, nullable=True),
    "police_district": pa.Column(pa.Category, coerce=True),
    "license_code": pa.Column(pa.Category, coerce=True),
    "license_description": pa.Column(str, coerce=True),
    "business_activity_id": pa.Column(str, coerce=True, nullable=True),
    "business_activity": pa.Column(pa.Category, coerce=True, nullable=True),
    "license_number": pa.Column(str, coerce=True),
    "application_type": pa.Column(pa.Category, coerce=True),
    "application_created_date": pa.Column(str, coerce=True, nullable=True),
    "application_requirements_complete": pa.Column(pa.DateTime, coerce=True),
    "payment_date": pa.Column(pa.DateTime, coerce=True),
    "conditional_approval": pa.Column(str, coerce=True),
    "license_start_date": pa.Column(pa.DateTime, coerce=True),
    "expiration_date": pa.Column(pa.DateTime, coerce=True),
    "license_approved_for_issuance": pa.Column(str, coerce=True),
    "date_issued": pa.Column(pa.DateTime, coerce=True),
    "license_status": pa.Column(pa.Category, coerce=True),
    "license_status_change_date": pa.Column(pa.DateTime, coerce=True, nullable=True),
    "ssa": pa.Column(str, coerce=True, nullable=True),
    "latitude": pa.Column(pa.Float, coerce=True, nullable=True, checks=[
        pa.Check.between(38, 44)
    ]),
    "longitude": pa.Column(pa.Float, coerce=True, nullable=True, checks=[
        pa.Check.between(-89, -84)
    ]),
    "location": pa.Column(str, coerce=True),
})

business_license_validated = business_license_schema.validate(business_license_raw)
print(business_license_validated.info())
business_license_validated

/Users/samedwardes/.pyenv/versions/3.11.1/lib/python3.11/site-packages/pandera/backends/pandas/array.py:167: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  f"non-nullable series '{check_obj.name}' contains "


SchemaError: non-nullable series 'ward' contains null values:
219    NaN
220    NaN
221    NaN
349    NaN
612    NaN
613    NaN
614    NaN
705    NaN
706    NaN
707    NaN
708    NaN
709    NaN
710    NaN
995    NaN
996    NaN
997    NaN
998    NaN
Name: ward, dtype: object

In [8]:
(business_license_raw["id"] == "1000929-20021116").value_counts()

False    998
True       2
Name: id, dtype: int64

In [9]:
business_license_raw[business_license_raw["id"] == "1000929-20021116"]

,id,license_id,account_number,site_number,legal_name,doing_business_as_name,address,city,state,zip_code,ward,precinct,ward_precinct,police_district,license_code,license_description,business_activity_id,business_activity,license_number,application_type,application_created_date,application_requirements_complete,payment_date,conditional_approval,license_start_date,expiration_date,license_approved_for_issuance,date_issued,license_status,license_status_change_date,ssa,latitude,longitude,location
945,1000929-20021116,1279618,6202,1,"LINCOLN PENNY, INC",3-PENNY CINEMA,2424-2426 N LINCOLN AVE,CHICAGO,IL,60614,43.0,56.0,43-56,19.0,1475,Consumption on Premises - Incidental Activity,638.0,Consumption of Liquor on Premises,1000929,RENEW,2002-09-24T00:00:00.000,2003-05-14T00:00:00.000,2002-11-19T00:00:00.000,N,2002-11-16T00:00:00.000,2003-05-15T00:00:00.000,2003-05-14T00:00:00.000,2003-05-14T00:00:00.000,AAI,NaN,35.0,41.925938,-87.649665,"\n, \n(41.92593792180545, -87.64966458497088)"
946,1000929-20021116,1355691,6202,1,"LINCOLN PENNY, INC",3-PENNY CINEMA,2424-2426 N LINCOLN AVE,CHICAGO,IL,60614,43.0,56.0,43-56,19.0,1475,Consumption on Premises - Incidental Activity,638.0,Consumption of Liquor on Premises,1000929,RENEW,NaN,2003-05-14T00:00:00.000,2003-05-14T00:00:00.000,N,2002-11-16T00:00:00.000,2003-11-15T00:00:00.000,2003-10-30T00:00:00.000,2003-10-30T00:00:00.000,AAI,NaN,35.0,41.925938,-87.649665,"\n, \n(41.92593792180545, -87.64966458497088)"


In [ ]:
# Pin the data to Connect
pin_name = f"{user_name}/chicago-business-license-data-validated"
board.pin_write(
    business_license_validated, 
    name=pin_name, 
    type="csv",
    versioned=True,
    title="City of Chicago - Business License Data (VALIDATED)"
)

## Data set (2): Food inspections

<https://data.cityofchicago.org/Health-Human-Services/Food-Inspections/4ijn-s7e5>

In [ ]:
pin_name = f"{user_name}/chicago-food-inspection-data"
food_inspection_raw = board.pin_read(pin_name)
food_inspection_raw

In [ ]:
food_inspection_schema = pa.DataFrameSchema({
    "inspection_id": pa.Column(str, coerce=True, unique=True),
    "dba_name": pa.Column(str, coerce=True),
    "aka_name": pa.Column(str, coerce=True, nullable=True),
    "license_": pa.Column(str, coerce=True),
    "facility_type": pa.Column(pa.Category, coerce=True, nullable=True),
    "risk": pa.Column(str, coerce=True),
    "address": pa.Column(str, coerce=True),
    "city": pa.Column(str, coerce=True, checks=[
        pa.Check.eq("CHICAGO")
    ]),
    "state": pa.Column(str, coerce=True),
    "zip": pa.Column(str, coerce=True),
    "inspection_date": pa.Column(pa.DateTime, coerce=True),
    "inspection_type": pa.Column(str, coerce=True),
    "results": pa.Column(str, coerce=True),
    "violations": pa.Column(str, coerce=True, nullable=True),
    "latitude": pa.Column(pa.Float, coerce=True, nullable=True, checks=[
        pa.Check.between(38, 44)
    ]),
    "longitude": pa.Column(pa.Float, coerce=True, nullable=True, checks=[
        pa.Check.between(-89, -84)
    ]),
    "location": pa.Column(str, coerce=True, nullable=True),
})

food_inspection_validated = food_inspection_schema.validate(food_inspection_raw)
print(food_inspection_validated.info())
food_inspection_validated

In [ ]:
# Pin the data to Connect
pin_name = f"{user_name}/chicago-food-inspection-data-validated"
board.pin_write(
    food_inspection_validated, 
    name=pin_name, 
    type="csv",
    versioned=True,
    title="City of Chicago - Food Inspection Data (VALIDATED)"
)

# Tmp

In [ ]:
# profile = ProfileReport(business_license_raw, title="Profiling Report")
# profile